In [3]:
import cv2
from IPython.display import clear_output
from matplotlib import pyplot as plt
from ultralytics import YOLO
import numpy as np
import heapq
import math
import time
%matplotlib inline

t0 = time.perf_counter()
def resize_wh(orig_w, orig_h):
    # h, w, d = image.shape
    imgsz = 1280 # image size of yolo default is 640
    new_h = imgsz/orig_w * orig_h # new height with same ratio
    w = imgsz
    remainder = (new_h % 32) # YOLO need w,h that can divide by 32
    if remainder > 32/2:
        h = int(new_h - remainder + 32)
    else:
        h = int(new_h - remainder)
    return (w,h)

vdo_path = 'testvid6.mov'
vdo = cv2.VideoCapture(vdo_path)

model_path = r'./runs/detect/train36-v8l/weights/best.pt'
model = YOLO(model_path)

width = int(vdo.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vdo.get(cv2.CAP_PROP_FRAME_HEIGHT))
width, height = resize_wh(width, height)
fps = vdo.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(filename='videoresult.mp4', fourcc=cv2.VideoWriter_fourcc(*'mp4v'), fps=fps, frameSize=(width, height))

start_node = (25,1250) #(1100, 300) #(10,10) #testvid2-(25,1250) #testvid4-(25,25)
goal_node = (1700,25) #(600, 800) #(1500,1200) #testvid2-(1700,25) #testvid4-(950,1275)

frame_count = 0
timels = []

frame_in_danger_radius_count = []
frame_in_obstacle_count = []

while vdo.isOpened():
    ret, frame = vdo.read()
    # print(frame_count)
    if ret:
        # detection part
        t0_det = time.perf_counter()
        # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # convert image to RGB color
        frame = cv2.resize(frame, (width, height))
        results = model.predict(frame, imgsz=(height, width), verbose=False)
        for result in results:
            box = result.boxes.xywh
        t1_det = time.perf_counter()

        if start_node == goal_node:
            final_img = result.plot(labels=False)
            for coor in box:
                cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius, (0,0,255), thickness=3)
                cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius+50, (255,0,0), thickness=2)
            for i in range(len(path)-1):
                cv2.line(final_img, (path[i][1],path[i][0]), (path[i+1][1],path[i+1][0]), (0,255,0), thickness=5)
            final_img = cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1)
            out.write(final_img)
            t0_show = time.perf_counter()
            # plt.imshow(cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1))
            # plt.show()
            t1_show = time.perf_counter()
            print('frame', frame_count)
            print(start_node)
            frame_count += 1
            clear_output(wait=True)
            timels.append([frame_count, t1_det-t0_det, t1_map-t0_map, t1_astar-t0_astar, t1_show-t0_show])
            continue

        # update new path every 2 frame passed
        if frame_count % 2 == 0:
            # map generation part
            t0_map = time.perf_counter()
            map = np.ones((height, width), dtype=np.uint16)
            obstacle_radius = 50
            danger_radius = 50
            thickness = -1

            for i in range(danger_radius):
                g_cost = (obstacle_radius+50)*i
                for coor in box:
                    cv2.circle(map, (int(coor[0]), int(coor[1])), obstacle_radius+danger_radius, (g_cost), thickness) #0, thickness) # (g_cost), thickness)
                danger_radius -= 1
            for coor in box:
                cv2.circle(map, (int(coor[0]), int(coor[1])), obstacle_radius, 0, thickness)
            t1_map = time.perf_counter()

            # count the frame that UAV enter danger radius
            if map[start_node[0]][start_node[1]] != 1 and map[start_node[0]][start_node[1]] != 0:
                frame_in_danger_radius_count.append(frame_count)

            # count the frame that UAV crash with obstacle
            if map[start_node[0]][start_node[1]] == 0:
                frame_in_danger_radius_count.append(frame_count)

            t0_astar = time.perf_counter()
            open_list = [(0, (start_node))]
            close_list = []
            cumulative_g_cost = {}
            cumulative_g_cost[start_node] = map[start_node[0]][start_node[1]]
            came_from = {}

            while open_list:
                _, current = heapq.heappop(open_list)
                close_list.append(current)
                # print(current)

                # early exit
                if current == goal_node:
                    path = [current]
                    while current in came_from:
                        # print(current)
                        current = came_from[current]
                        path.append(current)
                    path.reverse()
                    # print(path)
                    break

                # get neighbor node
                for step in [(0,-25), (0,25), (-25,0), (25,0), (25,25), (-25,25), (25,-25), (-25,-25)]: #(0,-1), (0,1), (-1,0), (1,0), (1,1), (-1,1), (1,-1), (-1,-1)
                    neighbor = (current[0] + step[0], current[1] + step[1])

                    # within the grid range
                    if neighbor[0] < 0 or neighbor[0] >= (map.shape[0]) or neighbor[1] < 0 or neighbor[1] >= map.shape[1]:
                        continue

                    # not step on the obstacle
                    if map[neighbor[0]][neighbor[1]] == 0:
                        continue

                    if neighbor in close_list:
                        continue

                    if neighbor in dict(open_list).values():
                        temp_g = map[neighbor[0]][neighbor[1]] + cumulative_g_cost[current]
                        if temp_g >= cumulative_g_cost[neighbor]:
                            continue
                    
                    cumulative_g_cost[neighbor] = map[neighbor[0]][neighbor[1]] + cumulative_g_cost[current]
                    g_cost = map[neighbor[0]][neighbor[1]]
                    h_cost = math.sqrt(((neighbor[0] - goal_node[0])**2) + ((neighbor[1] - goal_node[1])**2)) # euclidean distance
                    f_cost = g_cost + h_cost
                    
                    came_from[neighbor] = current
                    heapq.heappush(open_list, (f_cost, neighbor))
            t1_astar = time.perf_counter()

            # no path found will stay at the same place
            try:
                current_coor
            except NameError:
                current_coor = start_node
            try:
                path
            except NameError:
                path = current_coor

            current_coor = (path[0][1], path[0][0]) # (x, y)
            final_img = result.plot(labels=False)
            for coor in box:
                cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius, (0,0,255), thickness=3)
                cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius+50, (255,0,0), thickness=2)
            for i in range(len(path)-1):
                cv2.line(final_img, (path[i][1],path[i][0]), (path[i+1][1],path[i+1][0]), (0,255,0), thickness=5)
            final_img = cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1)
            out.write(final_img)
            if len(path) >= 2:
                start_node = path[1]
            t0_show = time.perf_counter()
            # plt.imshow(cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1))
            # plt.show()
            t1_show = time.perf_counter()
            print('frame', frame_count)
            print(path)
            frame_count += 1
            clear_output(wait=True)
            timels.append([frame_count, t1_det-t0_det, t1_map-t0_map, t1_astar-t0_astar, t1_show-t0_show])
            continue

        final_img = result.plot(labels=False)
        for coor in box:
            cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius, (0,0,255), thickness=3)
            cv2.circle(final_img, (int(coor[0]), int(coor[1])), obstacle_radius+50, (255,0,0), thickness=2)
        for i in range(len(path)-1):
            cv2.line(final_img, (path[i][1],path[i][0]), (path[i+1][1],path[i+1][0]), (0,255,0), thickness=5)
        final_img = cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1)
        out.write(final_img)
        t0_show = time.perf_counter()
        # plt.imshow(cv2.circle(final_img, current_coor, 20, color=(255, 16, 240),thickness=-1))
        # plt.show()
        t1_show = time.perf_counter()
        print('frame', frame_count)
        print(path)
        frame_count += 1
        clear_output(wait=True)
        timels.append([frame_count, t1_det-t0_det, t1_map-t0_map, t1_astar-t0_astar, t1_show-t0_show])
    else:
        break

t1 = time.perf_counter()
out.release()

frame 199
(1700, 25)


In [4]:
# sample
import pandas as pd
print('total time', t1 - t0)
df = pd.DataFrame(timels)
df.rename(columns = {0:'frame', 1:'detection_time', 2:'map_generation_time', 3:'a_star_time', 4:'visualize_time'}, inplace = True)
df.describe()

total time 24.922874912968837


,frame,detection_time,map_generation_time,a_star_time,visualize_time
count,200.000000,200.000000,200.000000,200.000000,2.000000e+02
mean,100.500000,0.055581,0.069971,0.005178,1.806749e-06
std,57.879185,0.034086,0.010279,0.005748,1.251049e-06
min,1.000000,0.051896,0.054306,0.000106,9.390060e-07
25%,50.750000,0.052751,0.057807,0.000423,1.383509e-06
50%,100.500000,0.053071,0.070076,0.003059,1.662527e-06
75%,150.250000,0.053399,0.079365,0.007995,2.027286e-06
max,200.000000,0.535057,0.094449,0.033339,1.808791e-05
